Colab Code Block 1: Install Dependencies

In [1]:
# Install necessary packages
!pip install streamlit
!pip install langchain
!pip install sentence-transformers
!pip install python-dotenv
!pip install langchain_groq
!pip install langchain_community
!pip install faiss-cpu  # FAISS for vector storage
!pip install evaluation

Colab Code Block 2: Import Libraries

In [2]:
import streamlit as st
import os
import json
from pathlib import Path
import glob
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer  # New embedding model
from langchain.embeddings import HuggingFaceEmbeddings  # Wrapper for SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from dotenv import load_dotenv
# from evaluation import evaluate_rag_response
import time

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
...
model_kwargs={"device": device}

In [3]:
from transformers import pipeline, set_seed
from transformers import BioGptTokenizer, BioGptForCausalLM
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

Colab Code Block 3: Load Environment & Initialize LLM + Embeddings

In [11]:
!pip install mistral-common
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.1 MB/s eta 0:00:00


In [26]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

In [27]:
# Ensure the API key is available
if not groq_api_key:
    raise ValueError("API Key is not set in the secrets.")

In [28]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")

In [34]:
# Using clinical-BERT for embeddings (no API needed)
embeddings = HuggingFaceEmbeddings(
    model_name="emilyalsentzer/Bio_ClinicalBERT",
    model_kwargs={"device": "cuda"}  # Use "cuda" if you have GPU
)

<ipython-input-34-5d7ed8594fd1>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [9]:
!unzip Diagnosis_flowchart.zip
!unzip Finished.zip

Archive:  Diagnosis_flowchart.zip
   creating: Diagnosis_flowchart/
  inflating: Diagnosis_flowchart/Acute Coronary Syndrome.json  
  inflating: Diagnosis_flowchart/Adrenal Insufficiency.json  
  inflating: Diagnosis_flowchart/Alzheimer.json  
  inflating: Diagnosis_flowchart/Aortic Dissection.json  
  inflating: Diagnosis_flowchart/Asthma.json  
  inflating: Diagnosis_flowchart/Atrial Fibrillation.json  
  inflating: Diagnosis_flowchart/Cardiomyopathy.json  
  inflating: Diagnosis_flowchart/COPD.json  
  inflating: Diagnosis_flowchart/Diabetes.json  
  inflating: Diagnosis_flowchart/Epilepsy.json  
  inflating: Diagnosis_flowchart/Gastro-oesophageal Reflux Disease.json  
  inflating: Diagnosis_flowchart/Heart Failure.json  
  inflating: Diagnosis_flowchart/Hyperlipidemia.json  
  inflating: Diagnosis_flowchart/Hypertension.json  
  inflating: Diagnosis_flowchart/Migraine.json  
  inflating: Diagnosis_flowchart/Multiple Sclerosis.json  
  inflating: Diagnosis_flowchart/Peptic Ulcer Dis

Colab Code Block 4: Load Clinical Documents

In [35]:
def load_clinical_data():
    """Load both flowcharts and patient cases"""
    docs = []

    # 1. Load diagnosis flowcharts
    for fpath in glob.glob("./Diagnosis_flowchart/*.json"):
        with open(fpath) as f:
            data = json.load(f)
            content = f"""
            DIAGNOSTIC FLOWCHART: {Path(fpath).stem}
            Diagnostic Path: {data['diagnostic']}
            Key Criteria: {data['knowledge']}
            """
            docs.append(Document(
                page_content=content,
                metadata={"source": fpath, "type": "flowchart"}
            ))

    # 2. Load patient cases
    for category_dir in glob.glob("./Finished/*"):
        if os.path.isdir(category_dir):
            for case_file in glob.glob(f"{category_dir}/*.json"):
                with open(case_file) as f:
                    case_data = json.load(f)
                    notes = "\n".join(
                        f"{k}: {v}" for k, v in case_data.items()
                        if k.startswith("input")
                    )
                    docs.append(Document(
                        page_content=f"""
                        PATIENT CASE: {Path(case_file).stem}
                        Category: {Path(category_dir).name}
                        Notes: {notes}
                        """,
                        metadata={"source": case_file, "type": "patient_case"}
                    ))
    return docs

In [36]:
docs = load_clinical_data()
print(f"Loaded {len(docs)} documents")

Loaded 192 documents


In [37]:
print(docs[0].page_content)


            DIAGNOSTIC FLOWCHART: Pulmonary Embolism
            Diagnostic Path: {'Suspected Pulmonary Embolism': {'Pulmonary Embolism': {'Massive PE': [], 'Submassive PE': [], 'Low-risk PE': []}}}
            Key Criteria: {'Suspected Pulmonary Embolism': {'Risk Factors': 'HTN; Previous VTE; Immobility or recent surgery; Cancer; Thrombophilia; Hormonal therapy (e.g., oral contraceptives or hormone replacement therapy); Pregnancy and the postpartum period; Obesity; Smoking; Long travel history.; etc.', 'Symptoms': 'Sudden onset of dyspnea; Chest pain (sharp and worsened with deep breaths); Hemoptysis; Syncope or dizziness; Tachypnea; Tachycardia; etc.', 'Signs': 'Tachypnea (rapid breathing); Tachycardia (rapid heart rate); Hypoxia (low oxygen levels in the blood); Cyanosis (blueish coloration of the skin and lips); Fever; Signs of deep vein thrombosis (DVT), such as swelling, redness, or pain in the leg.; etc.'}, 'Pulmonary Embolism': "Multi-slice spiral CT (CTPA): directly displays 

Colab Code Block 5: Vectorstore Builder

In [38]:
def build_vectorstore():
    documents = load_clinical_data()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(splits, embeddings)
    return vectorstore

In [39]:
vectorstore = build_vectorstore()
print("✅ Vectorstore built successfully.")

✅ Vectorstore built successfully.


Colab Code Block 6: Define RAG Chain

In [56]:
def run_rag_chat(query, vectorstore):
    retriever = vectorstore.as_retriever()

    prompt_template = ChatPromptTemplate.from_template("""
    You are a clinical assistant AI. Based on the following clinical context, provide a reasoned and medically sound answer to the question.

    <context>
    {context}
    </context>

    Question: {input}

    Answer:
    """)
    retrieved_docs = retriever.invoke(query, k=3)
        # Extract the content of the retrieved documents
    retrieved_context = "\n".join([doc.page_content for doc in retrieved_docs])

    chain = create_retrieval_chain(
        retriever,
        create_stuff_documents_chain(llm, prompt_template)
    )

    # response = chain.invoke({"input": query})
    response = chain.invoke({"input": query, "context": retrieved_context})

    return response

Colab Code Block 7: Evaluation Logic

In [58]:
from sentence_transformers import util
import numpy as np

def calculate_hit_rate(retriever, query, expected_docs, k=3):
    """
    Custom hit rate calculation for top-k retrieved documents.
    Args:
        retriever: FAISS retriever (st.session_state.vectors.as_retriever())
        query: User's input query
        expected_docs: List of expected document contents
        k: Top-k documents to consider
    Returns:
        hit_rate: Percentage of expected docs found in top-k results
    """
    retrieved_docs = retriever.get_relevant_documents(query, k=k)
    retrieved_contents = [doc.page_content for doc in retrieved_docs]

    hits = 0
    for expected in expected_docs:
        if any(expected in retrieved for retrieved in retrieved_contents):
            hits += 1

    return hits / len(expected_docs) if expected_docs else 0.0

def evaluate_rag_response(response, embeddings):
    scores = {}

    # 1. Faithfulness: Answer-Context Similarity
    answer_embed = embeddings.embed_query(response["answer"])
    context_embeds = [embeddings.embed_query(doc.page_content) for doc in response["context"]]
    similarities = [util.cos_sim(answer_embed, ctx_embed).item() for ctx_embed in context_embeds]
    scores["faithfulness"] = float(np.mean(similarities)) if similarities else 0.0

    # 2. Custom Hit Rate Calculation
    retriever = response["retriever"]  # Assuming retriever is passed as part of the response object
    scores["hit_rate"] = calculate_hit_rate(
        retriever,
        query=response["input"],
        expected_docs=[doc.page_content for doc in response["context"]],
        k=3
    )

    return scores

 Colab Code Block 8: Run Chat & View Result

In [57]:
user_input = input("🩺 Ask a clinical question: ")

response = run_rag_chat(user_input, vectorstore)
response["retriever"] = vectorstore.as_retriever()

print("\n💬 Answer:")
print(response['answer'])

print("\n📄 Sources:")
for doc in response["context"]:
    print(f"- {doc.metadata['source']}")
    print(f"  → {doc.page_content[:200]}...\n")

# Optional: Evaluation
try:
    eval_scores = evaluate_rag_response(response, embeddings)
    print("\n📊 Evaluation:")
    print(f"• Hit Rate (Top-3): {eval_scores['hit_rate']:.2f}")
    print(f"• Faithfulness: {eval_scores['faithfulness']:.2f}")
except Exception as e:
    print(f"⚠️ Evaluation failed: {str(e)}")


🩺 Ask a clinical question: What are the differential diagnoses for a patient with COPD presenting with increased shortness of breath?

💬 Answer:
For a patient with Chronic Obstructive Pulmonary Disease (COPD) presenting with increased shortness of breath, the differential diagnoses can be broad and varied, given the complexity of COPD and its potential for exacerbations and complications. Here are some key considerations:

1. **COPD Exacerbation**: This is the most immediate concern for a patient with COPD presenting with worsening symptoms. Exacerbations can be triggered by respiratory infections, air pollution, or other factors and are characterized by an increase in symptoms such as shortness of breath, cough, and sputum production.

2. **Pneumonia**: Patients with COPD are at an increased risk of developing pneumonia, which can present with worsening shortness of breath, cough, fever, and chest pain.

3. **Pulmonary Embolism (PE)**: Although less common, PE is a significant and pot